In [2]:
from tensorflow.keras import datasets, layers, models

import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mlp
import time

In [4]:
def padImage(image, pixels=5):
    bottom = image[-pixels:]
    top = image[:pixels]

    img = np.insert(image, 0, bottom, 0)
    img = np.insert(img, len(img), top, 0)
    img = np.insert(img, [0], [0] * pixels, 1)
    img = np.insert(img, [-1], [0] * pixels, 1)
    return img

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = (train_images / 255.0).astype(np.float32), test_images.astype(np.float32) / 255.0
test = np.loadtxt('mnist_rotation_new/mnist_all_rotation_normalized_float_test.amat')

In [ ]:
### THE PAD HAS TO BE DONE IN THE
### POLAR SPACE

# 20 is the ceiling of (14 * sqrt(2))
X_train_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in train_images]
X_train_polar = [padImage(x, pixels=5) for x in X_train_polar]
X_train_polar = np.array(X_train_polar)[...,None]

X_test_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in test_images]
X_test_polar = [padImage(x, pixels=5) for x in X_test_polar]
X_test_polar = np.array(X_test_polar)[...,None]

# Rotate test set
X_test_r_polar = [tfa.image.rotate(x, np.random.uniform(-np.pi/2., np.pi/2.)).numpy() for x in test_images]
X_test_r_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in X_test_r_polar]
X_test_r_polar = [padImage(x, pixels=5) for x in X_test_r_polar]
X_test_r_polar = np.array(X_test_r_polar)[...,None]

In [3]:
model = models.Sequential()
model.add(layers.Input(shape=X_train_polar.shape[1:]))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dense(64, activation='linear'))
model.add(layers.Activation('relu'))
model.add(layers.Dense(10))

NameError: name 'X_train_polar' is not defined

In [86]:
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
# checkpoint = tf.keras.callbacks.ModelCheckpoint(name + '.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')
# es = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', min_delta=0.000, patience=10, verbose=1,
#     mode='auto', baseline=None, restore_best_weights=False
# )


# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
# times = timecallback()

# model.fit(X_train_polar, train_labels, batch_size=128, epochs=100, 
#           validation_data=(X_test_r_polar, test_labels),
#           callbacks=[reduce_lr, checkpoint, es, times])

In [30]:
from datetime import datetime

class timecallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.times = []
        self.epochs = []
        # use this value as reference to calculate cummulative time taken
        self.timetaken = tf.timestamp()
    def on_epoch_end(self,epoch,logs = {}):
        self.times.append(tf.timestamp() - self.timetaken)
        self.epochs.append(epoch)
    def on_train_end(self,logs = {}):
        plt.xlabel('Epoch')
        plt.ylabel('Total time taken until an epoch in seconds')
        plt.plot(self.epochs, self.times, 'ro')
        for i in range(len(self.epochs)):
          j = self.times[i].numpy()
          if i == 0:
            plt.text(i, j, str(round(j, 3)))
          else:
            j_prev = self.times[i-1].numpy()
            plt.text(i, j, str(round(j-j_prev, 3)))
        #plt.savefig(datetime.now().strftime("%Y%m%d%H%M%S") + ".png")

### Class wise accuracy dist.

In [51]:
class_acc = []
preds_dict = {}
for i in range(10):
    preds_dict[i] = m.predict(X_test_r_polar[np.where(test_labels==i)]).argmax(axis=1)
    class_acc.append(np.mean(preds_dict[i] == \
          test_labels[np.where(test_labels==i)])*100)
class_acc = np.asarray(class_acc)

In [52]:
class_acc

array([98.67346939, 98.41409692, 92.34496124, 93.96039604, 91.03869654,
       86.43497758, 73.9039666 , 71.9844358 , 91.99178645, 70.96134787])

In [56]:
np.average(class_acc[np.where(class_acc >= 75)])

93.26548344968012

In [61]:
unique, counts = np.unique(preds_dict[6], return_counts=True)
dict(zip(unique, counts))

{0: 11, 1: 1, 2: 7, 3: 9, 4: 7, 5: 119, 6: 708, 7: 8, 8: 2, 9: 86}

# Start

In [222]:
# No need to train just load model
name = 'm_pol_mnist_layerAdjust'

In [223]:
# Plain accuracy
m = tf.keras.models.load_model(name+'.h5')
accuracy = np.mean(m.predict(X_test_polar).argmax(axis=1) == test_labels)*100
# "Rotated" accuracy
accuracy_rotated = np.mean(m.predict(X_test_r_polar).argmax(axis=1) == test_labels)*100

print(accuracy, accuracy_rotated)

98.3 87.11


### Compute 12 angles (30 deg.) for each image in test image and store/load

In [85]:
model2f = models.Model(m.inputs, m.layers[-3].output)
# all elements in the test, and every 12 degrees, should have 30* all elements in the set
# save them acording to the degrees, 30 degrees, 30 files, each file contains 10000*64 

In [128]:
# 360/12 = 30
samplings = 12
rad_d30 = (30/360)*(2*np.pi)
a = [i * rad_d30 for i in range(360//30)]

In [66]:
def prep_images(images, angle, pad_pixels=5):
    processed_imgs = [tfa.image.rotate(x, angle).numpy() for x in images]
    processed_imgs = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in processed_imgs]
    processed_imgs = [padImage(x, pixels=pad_pixels) for x in processed_imgs]
    processed_imgs = np.array(processed_imgs)[...,None]
    return processed_imgs

In [81]:
have_file = True
file_name = 'all_MNIST_test_imgs_12samples_30deg_relu_activations.npz'
if not have_file:
    angle_preds = []
    for i in range(samplings):
        imgs = prep_images(test_images, a[i], pad_pixels=5)
        angle_preds.append(model2f.predict(imgs))
    angle_preds = np.asarray(angle_preds)
    np.savez(file_name, angle_preds)
else:
    angle_preds = np.load(file_name)['arr_0']

### Comparing average dist. of a single images average dist. across the twelve angles, with just the distances across different images in the test set

In [217]:
# Doing for class 4
class_idx = 4
class_entries = np.where(test_labels == class_idx)[0]

# extracting all the 4 images from each angular sample
class_extr = angle_preds[:,[class_entries]]
class_extr = np.reshape(class_extr, (samplings, len(class_entries), 64))
# checking if extractions are correct
for i in range(samplings):
    if not (class_extr[i] == angle_preds[i][class_entries]).all():
        print('not working')

### Computing distance for each image across its 12 angular rotations

In [218]:
class_12 = []
for i in range(len(class_entries)):
    # all angles for a single image / 64 activations instead of 1
    pred = class_extr[:, i]
    d = np.sum((pred[:,None] - pred[None])**2, axis=-1)
    d = np.maximum(d, 0)**.5
    n = pred.shape[0]
    class_12.append(np.sum(d)/(n*(n-1)/2))
class_12 = np.asarray(class_12)

In [219]:
np.std(class_12), np.average(class_12), np.median(class_12)

(3.6958994913965735, 42.249743213876044, 42.39877411813447)

### Computing Euclidean dist. between all images in the for every class (randomly rotated)

In [220]:
class_all = []
for i in range(10):
    preds_all = model2f.predict(X_test_r_polar[np.where(test_labels == i)])
    
    d = np.sum((preds_all[:,None] - preds_all[None])**2, axis=-1)
    d = np.maximum(d, 0)**.5
    n = preds_all.shape[0]
    class_all.append(np.sum(d)/(n*(n-1)/2))
class_all = np.asarray(class_all)

In [221]:
class_all

array([44.01405432, 36.69566852, 56.40035639, 56.16089649, 53.60868726,
       54.38765835, 59.14449949, 54.38713491, 54.93994948, 56.02892617])